# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pangody,65.8500,74.4859,19.54,40,100,7.58,RU,1720528808
1,1,bethel,41.3712,-73.4140,26.04,90,0,1.54,US,1720528428
2,2,kapuskasing,49.4169,-82.4331,16.05,100,100,4.12,CA,1720528560
3,3,shancheng,34.7953,116.0817,24.05,82,37,1.85,CN,1720528811
4,4,ashland,40.8167,-82.2665,24.04,97,100,1.45,US,1720528812


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    title = "World Map - Humidity"
)

# Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Humidity'] <= 50) &
    (city_data_df['Wind Speed'] <= 5) &
    (city_data_df['Max Temp'] >= 20) & 
    (city_data_df['Max Temp'] <= 30) & 
    (city_data_df['Cloudiness'] <= 30)
]
# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,vorgashor,67.5833,63.9500,24.16,30,11,0.64,RU,1720528431
75,75,revda,56.8010,59.9303,25.34,46,23,3.33,RU,1720528898
108,108,rakops,-21.0167,24.3333,20.96,6,0,2.22,BW,1720528937
151,151,salekhard,66.5300,66.6019,25.35,41,10,1.26,RU,1720528992
163,163,alvand,36.3187,49.1678,23.32,38,8,4.50,IR,1720529185
178,178,yakeshi,49.2833,120.7333,22.23,34,25,1.22,CN,1720529309
200,200,nar'yan-mar,67.6713,53.0870,27.15,50,3,3.31,RU,1720529339
205,205,gornopravdinsk,60.0500,69.9000,26.24,45,29,2.62,RU,1720529345
231,231,iskateley,67.6803,53.1512,27.10,50,5,3.36,RU,1720529378
250,250,tsiombe,-25.3000,45.4833,26.33,38,0,4.93,MG,1720529401


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
61,vorgashor,RU,67.5833,63.9500,30,
75,revda,RU,56.8010,59.9303,46,
108,rakops,BW,-21.0167,24.3333,6,
151,salekhard,RU,66.5300,66.6019,41,
163,alvand,IR,36.3187,49.1678,38,
178,yakeshi,CN,49.2833,120.7333,34,
200,nar'yan-mar,RU,67.6713,53.0870,50,
205,gornopravdinsk,RU,60.0500,69.9000,45,
231,iskateley,RU,67.6803,53.1512,50,
250,tsiombe,MG,-25.3000,45.4833,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vorgashor - nearest hotel: София
revda - nearest hotel: Металлург
rakops - nearest hotel: Rakops River Lodge
salekhard - nearest hotel: No hotel found
alvand - nearest hotel: هتل لاله هیدج
yakeshi - nearest hotel: Baoling Hotel
nar'yan-mar - nearest hotel: Гостиница "Агат"
gornopravdinsk - nearest hotel: Гостинный двор
iskateley - nearest hotel: Авантаж
tsiombe - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
shakawe - nearest hotel: EdenWood guesthouse
humaita - nearest hotel: Humaita Quality Hotel
sosnogorsk - nearest hotel: Гостиница Жемчужина
asipovichy - nearest hotel: Асіповічы
bor - nearest hotel: Олимп гостиница
camacupa - nearest hotel: No hotel found
velikiy ustyug - nearest hotel: Сухона
mwinilunga - nearest hotel: No hotel found
guanambi - nearest hotel: Hotel Recife
mountain home - nearest hotel: Best Western Foothills Inn
pahrump - nearest hotel: Holiday Inn Express & Suites
tuba city - nearest hotel: Quality Inn Navajo N

,City,Country,Lat,Lng,Humidity,Hotel Name
61,vorgashor,RU,67.5833,63.9500,30,София
75,revda,RU,56.8010,59.9303,46,Металлург
108,rakops,BW,-21.0167,24.3333,6,Rakops River Lodge
151,salekhard,RU,66.5300,66.6019,41,No hotel found
163,alvand,IR,36.3187,49.1678,38,هتل لاله هیدج
178,yakeshi,CN,49.2833,120.7333,34,Baoling Hotel
200,nar'yan-mar,RU,67.6713,53.0870,50,"Гостиница ""Агат"""
205,gornopravdinsk,RU,60.0500,69.9000,45,Гостинный двор
231,iskateley,RU,67.6803,53.1512,50,Авантаж
250,tsiombe,MG,-25.3000,45.4833,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    hover_cols=['City', 'Country', 'Hotel Name'],
    title='Vacation Cities with Nearest Hotels'
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)